In [ ]:
import google.colab
google.colab.drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
from pathlib import Path
import shutil
import re
from tqdm.notebook import tqdm
import nltk
from nltk.corpus import stopwords
from tqdm.notebook import tqdm
import numpy as np
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
import numpy as np
from scipy.special import softmax

In [ ]:
path_folder = Path('/content/drive/MyDrive/shakiba')
name_data = 'tweets_2015_2019_cleaned_EN.csv'
tweet_data = pd.read_csv(path_folder/name_data)
len(tweet_data)

8691249

In [ ]:
length  = int(len(tweet_data)/3)
slice_n = 3
slice_data = tweet_data[(slice_n-1)*length: ]
print(f'row {(slice_n-1)*length} to {(slice_n-1)*length+len(slice_data)}')
print(f'{len(slice_data)=}')

row 5794166 to 8691249
len(slice_data)=2897083


In [ ]:
MODEL = f"cardiffnlp/twitter-xlm-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = TFAutoModelForSequenceClassification.from_pretrained(MODEL)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/841 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

tf_model.h5:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFXLMRobertaForSequenceClassification.

All the layers of TFXLMRobertaForSequenceClassification were initialized from the model checkpoint at cardiffnlp/twitter-xlm-roberta-base-sentiment.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFXLMRobertaForSequenceClassification for predictions without further training.


In [ ]:
try:
  with open(path_folder/f'slice_{slice_n}.txt','r') as file:
    for r in reversed(file.readlines()):
      if r.find('saved.') > -1:
        last_iter =int( re.findall(r'\d+',r)[0])
        last_row =int( re.findall(r'\d+',r)[1])
        break
except Exception as e :
  print(e)
  last_iter = 0
  last_row = 0
print(f'{last_row=} {last_iter=}')

try:
   all_scores = [np.load(path_folder/f'slice_{slice_n}.npy')]
except Exception as e:
  print(e)
  all_scores = []
print(f'{len(all_scores) = }')


batch_size = 1024
iter = last_iter

for i in tqdm(range(last_row,len(slice_data), batch_size)):
    iter += 1
    try:
        encoded_input = tokenizer(slice_data['text'][i:i+batch_size].to_list(),
                                  return_token_type_ids=False,  # This tokenizer does not take token_type_ids
                                  padding=True,  # Pad the sequences to the same length
                                  truncation=True,  # Truncate sequences longer than the model's max input length
                                  return_tensors='tf',  # Return TensorFlow tensors
                                  max_length=512)

        output = model(**encoded_input)
        scores = output[0].numpy()
        index = np.array(range(i,i+batch_size)).reshape((-1,1))
        scores = np.concat([index, softmax(scores, axis=1)], axis=1)
        all_scores.append(scores)

    except:
        with open(path_folder/f'slice_{slice_n}.txt','a') as f:
          f.write(f'{i}-{i+batch_size} is not tokenned. \n')
          print(f'{i}-{i+batch_size} is not tokenned. ')
        continue
      # Append scores to the list

    if iter%100 == 0:
      concatenated_scores = np.concatenate(all_scores, axis=0)
      np.save(path_folder/f'slice_{slice_n}.npy', concatenated_scores)
      with open(path_folder/f'slice_{slice_n}.txt','a') as f:
          f.write(f'{iter=} , {i+batch_size} is saved. \n')
          print(f'{iter=} ,{i+batch_size} is saved. ')

# Concatenate all scores into a single NumPy array
concatenated_scores = np.concatenate(all_scores, axis=0)
np.save(path_folder/f'slice_{slice_n}_finall.npy', concatenated_scores)
with open(path_folder/f'slice_{slice_n}.txt','a') as f:
          f.write(f'Done \n')
          print('Done')

[Errno 2] No such file or directory: '/content/drive/MyDrive/shakiba/slice_3.txt'
last_row=0 last_iter=0
[Errno 2] No such file or directory: '/content/drive/MyDrive/shakiba/slice_3.npy'
len(all_scores) = 0


  0%|          | 0/2830 [00:00<?, ?it/s]

94208-95232 is not tokenned. 
iter=100 ,102400 is saved. 
iter=200 ,204800 is saved. 
iter=300 ,307200 is saved. 
iter=400 ,409600 is saved. 
iter=500 ,512000 is saved. 
528384-529408 is not tokenned. 
547840-548864 is not tokenned. 
587776-588800 is not tokenned. 
595968-596992 is not tokenned. 
iter=600 ,614400 is saved. 
641024-642048 is not tokenned. 
669696-670720 is not tokenned. 
iter=700 ,716800 is saved. 
iter=800 ,819200 is saved. 
iter=900 ,921600 is saved. 
953344-954368 is not tokenned. 
iter=1000 ,1024000 is saved. 
iter=1100 ,1126400 is saved. 
iter=1200 ,1228800 is saved. 
iter=1300 ,1331200 is saved. 
1344512-1345536 is not tokenned. 
1358848-1359872 is not tokenned. 
1418240-1419264 is not tokenned. 
iter=1400 ,1433600 is saved. 
1440768-1441792 is not tokenned. 
iter=1500 ,1536000 is saved. 
iter=1600 ,1638400 is saved. 
iter=1700 ,1740800 is saved. 
iter=1800 ,1843200 is saved. 
iter=1900 ,1945600 is saved. 
iter=2000 ,2048000 is saved. 
iter=2100 ,2150400 is saved.